In [115]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import torchvision
import torchvision.transforms.v2 as transforms
# from torchmetrics importo
from torcheval.metrics import MulticlassAccuracy
from torcheval.metrics.functional import multiclass_accuracy
from torch.ao.quantization import QuantStub, DeQuantStub

In [47]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [48]:
class DatasetWrapper(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        # print(x)
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

In [49]:
transform_train = transforms.Compose([
    transforms.ColorJitter(),
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# trainset = torchvision.datasets.CIFAR10(
#     root='./CIFAR10', train=True, download=True, transform=transform_train)
trainset = torchvision.datasets.CIFAR10(
    root='./CIFAR10', train=True, download=True)

# split the train set into two
train_set_size = int(len(trainset) * 0.8)
valid_set_size = len(trainset) - train_set_size

seed = torch.Generator().manual_seed(42)
trainset, validset = torch.utils.data.random_split(trainset, [train_set_size, valid_set_size], generator=seed)

trainset = DatasetWrapper(trainset, transform_train)
validset = DatasetWrapper(validset, transform_test)

# Create train dataloader
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=24)
# Create validation dataloader
validloader = torch.utils.data.DataLoader(
    validset, batch_size=512, shuffle=False, num_workers=24)

testset = torchvision.datasets.CIFAR10(
    root='./CIFAR10', train=False, download=True, transform=transform_test)
# Create test dataloader
testloader = torch.utils.data.DataLoader(
    testset, batch_size=512, shuffle=False, num_workers=24)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


/home/semar/.local/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [50]:
class SeparableConv2d(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2d, self).__init__()
        self.dw_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.dw_bn = nn.BatchNorm2d(in_channels)
        self.pw_conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.pw_bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = F.relu(self.dw_bn(self.dw_conv(x)))
        out = F.relu(self.pw_bn(self.pw_conv(out)))
        return out


class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        # padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(in_planes, out_planes, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_planes, momentum=0.1),
            # Replace with ReLU
            nn.ReLU(inplace=False)
        )

In [100]:
class MyMobileNet(pl.LightningModule):
    cfg = [
        (32, 64, 1), 
        (64, 128, 2), 
        (128, 128, 1), 
        (128, 256, 2),
        (256, 256, 1),
        (256, 512, 2),
        (512, 512, 1),
        (512, 512, 1),
        (512, 512, 1),
        (512, 512, 1),
        (512, 512, 1),
        (512, 1024, 2),
        (1024, 1024, 1),
    ]
    
    def __init__(self, num_classes=10, alpha: float = 1):
        super(MyMobileNet, self).__init__()
        conv_out = int(32 * alpha)
        self.conv = nn.Conv2d(3, conv_out, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(conv_out)
        # self.accuracy = Accuracy("multiclass", num_classes=num_classes)
        # self.accuracy = MulticlassAccuracy()
        self.accuracy = multiclass_accuracy

        self.features = self.make_feature_extractor(alpha)
        self.linear = nn.Linear(1024, num_classes)

    def make_feature_extractor(self, alpha):
        layer_values = [(int(inp*alpha), int(out*alpha), chan) for inp, out, chan in self.cfg]
        layers = nn.Sequential(*[SeparableConv2d(*tup) for tup in layer_values])
        return layers

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.features(x)
        x = F.avg_pool2d(x, 2)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x
        
    def step(self, batch, batch_idx):
    # def training_step(self):
        x, y = batch
        logits = self.forward(x)
        loss = self.compute_loss(logits, y)
        acc = self.accuracy(logits, y)
        return loss, acc

    def training_step(self, batch, batch_idx):
    # def training_step(self):
        loss, acc = self.step(batch, batch_idx)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_accuracy", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
        
    def validation_step(self, batch, batch_idx):
    # def training_step(self):
        loss, acc = self.step(batch, batch_idx)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_accuracy", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
        
    def test_step(self, batch, batch_idx):
        loss, acc = self.step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_accuracy", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
        
    def configure_optimizers(self):
        self.lr = 0.02089
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.001, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.3, total_steps=MAX_EPOCHS * len(trainloader))
        return { "optimizer": optimizer, "lr_scheduler": scheduler }
        
    def compute_loss(self, logits, labels):
        return nn.functional.cross_entropy(logits, labels)

    # Fuse Conv+BN and Conv+BN+Relu modules prior to quantization
    # This operation does not change the numerics
    # def fuse_model(self, is_qat=False):
    #     fuse_modules = torch.ao.quantization.fuse_modules_qat if is_qat else torch.ao.quantization.fuse_modules
    #     for m in self.modules():
    #         print(m)
    #         if type(m) == SeparableConv2d:
    #             fuse_modules(m, ['0', '1', '2'], inplace=True)
            # if type(m) == InvertedResidual:
            #     for idx in range(len(m.conv)):
            #         if type(m.conv[idx]) == nn.Conv2d:
            #             fuse_modules(m.conv, [str(idx), str(idx + 1)], inplace=True)

In [95]:
# from lightning.pytorch.tuner import Tuner
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar

MAX_EPOCHS = 50

model = MyMobileNet()

checkpoint_callback = ModelCheckpoint(
    # monitor='val_accuracy',
    dirpath='checkpoints/',
    filename='ORIGINAL2-{epoch:02d}-{val_accuracy:.2f}'
)
progress_callback = RichProgressBar(leave=True)

trainer = pl.Trainer(max_epochs=MAX_EPOCHS, callbacks=[checkpoint_callback, progress_callback])

# tune = Tuner(trainer)
# optimal_lr = tune.lr_find(model, train_dataloaders=trainloader, val_dataloaders=validloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [96]:
trainer.fit(model, train_dataloaders=trainloader, val_dataloaders=validloader)

/home/semar/.local/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory checkpoints/ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name     ┃ Type        ┃ Params ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ conv     │ Conv2d      │    864 │
│ 1 │ bn       │ BatchNorm2d │     64 │
│ 2 │ features │ Sequential  │  3.2 M │
│ 3 │ linear   │ Linear      │ 10.2 K │
└───┴──────────┴─────────────┴────────┘

Trainable params: 3.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 3.2 M                                                                                                
Total estimated model params size (MB): 12

Output()

NameError: name 'out' is not defined

In [106]:
# quantized_model.to("cuda")
trainer.test(model, testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.826200008392334     │
│         test_loss         │     0.540178656578064     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.540178656578064, 'test_accuracy': 0.826200008392334}]

In [57]:
print('==> saving model at epoch 50')
state = {
    'net': model.state_dict(),
    'acc': '0.747299',
    'epoch': 50,
}
# torch.save(state, './checkpoints/original-50-test_acc=0.82.pth')

==> saving model at epoch 50


# Quantization!

In [119]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

class QuantizedMobileNet(MyMobileNet):
    def __init__(self, num_classes=10, alpha: float = 1):
        super(QuantizedMobileNet, self).__init__(num_classes, alpha)


    # Fuse Conv+BN and Conv+BN+Relu modules prior to quantization
    # This operation does not change the numerics
    def fuse_model(self, is_qat=False):
        fuse_modules = torch.ao.quantization.fuse_modules_qat if is_qat else torch.ao.quantization.fuse_modules
        for m in self.modules():
            if type(m) == QuantizedMobileNet:
                fuse_modules(m, ["conv", "bn"], inplace=True)
            if type(m) == SeparableConv2d:
                # print(f"Separable: {m}")
                fuse_modules(m, ["dw_conv", "dw_bn"], inplace=True)
                fuse_modules(m, ["pw_conv", "pw_bn"], inplace=True)
            # if type(m) == InvertedResidual:
            #     for idx in range(len(m.conv)):
            #         if type(m.conv[idx]) == nn.Conv2d:
            #             fuse_modules(m.conv, [str(idx), str(idx + 1)], inplace=True)

In [120]:
# Baseline
model = QuantizedMobileNet()
saved = torch.load("checkpoints/ORIGINAL2-epoch=49-val_accuracy=0.83.ckpt")
model.load_state_dict(saved['state_dict'])
print_size_of_model(model)

progress_callback = RichProgressBar(leave=True)

trainer = pl.Trainer(callbacks=[progress_callback])

trainer.test(model, testloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Size (MB): 12.999154


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.826200008392334     │
│         test_loss         │    0.5401778221130371     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5401778221130371, 'test_accuracy': 0.826200008392334}]

In [121]:
model = QuantizedMobileNet()
saved = torch.load("checkpoints/ORIGINAL2-epoch=49-val_accuracy=0.83.ckpt")
model.load_state_dict(saved['state_dict'])
model.eval()
model.fuse_model()
print_size_of_model(model)

progress_callback = RichProgressBar(leave=True)

trainer = pl.Trainer(callbacks=[progress_callback])

trainer.test(model, testloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Size (MB): 12.841274


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8263999819755554     │
│         test_loss         │    0.5401804447174072     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5401804447174072, 'test_accuracy': 0.8263999819755554}]

In [133]:
model = QuantizedMobileNet()
checkpoint = torch.load("checkpoints/ORIGINAL2-epoch=49-val_accuracy=0.83.ckpt")
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model.fuse_model()
model.qconfig = torch.ao.quantization.default_qconfig
print(model.qconfig)
torch.ao.quantization.prepare(model, inplace=True)

# Calibrate first
print('Post Training Quantization Prepare: Inserting Observers')
print('\n SeparableConv Block:After observer insertion \n\n', model.features[1].dw_conv)

trainer = pl.Trainer(callbacks=[progress_callback])

trainer.test(model, testloader)

torch.ao.quantization.convert(model, inplace=True)

print('Post Training Quantization: Convert done')
print('\n SeparableConv Block: After fusion and quantization, note fused modules: \n\n', model.features[1].dw_conv)
print_size_of_model(model)

trainer = pl.Trainer(callbacks=[progress_callback])

trainer.test(model, testloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})
Post Training Quantization Prepare: Inserting Observers

 SeparableConv Block:After observer insertion 

 Conv2d(
  64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64
  (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
)


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8263999819755554     │
│         test_loss         │    0.5401804447174072     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Post Training Quantization: Convert done

 SeparableConv Block: After fusion and quantization, note fused modules: 

 QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), scale=0.37764397263526917, zero_point=58, padding=(1, 1), groups=64)
Size (MB): 3.272438


Output()

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

QuantizedCPU: registered at ../aten/src/ATen/native/quantized/cpu/qconv.cpp:1874 [kernel]
Qua